In [1]:
import pandas as pd
import sqlite3

## 1. Use the sqlite3 library to create a connection to the database.


In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## 2. Using one query per group, create two dataframes, test_results and control_results, with the columns "time" and "avg_diff" and two rows.
- The "time" column should contain the values "after" and "before".
- The "avg_diff" should contain the average delta for all users for the time period before and after their first visit to the page.
- Only take into account users with observations before and after.
## 3. We are still not using the lab project1.

In [3]:
query_test_results = """
SELECT 
    CASE 
        WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
        ELSE 'after' END AS time_period,
        AVG((CAST(strftime('%s',t.first_commit_ts) AS INTEGER ) - d.deadlines ) / 3600) AS avg_diff
FROM
    test t
JOIN
    deadlines d ON t.labname = d.labs 
WHERE
    t.labname != 'project1'
AND
    t.first_commit_ts IS NOT NULL
AND
    t.first_view_ts IS NOT NULL
GROUP BY
    time_period
"""
test_results = pd.read_sql_query(query_test_results, conn)
test_results

,time_period,avg_diff
0,after,-103.40625
1,before,-60.56250


In [4]:
query_control_results = """
SELECT 
    CASE 
        WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
        ELSE 'after' END AS time_period,
        AVG((CAST(strftime('%s',t.first_commit_ts) AS INTEGER ) - d.deadlines ) / 3600) AS avg_diff
FROM
    control t
JOIN
    deadlines d ON t.labname = d.labs 
WHERE
    t.labname != 'project1'
AND
    t.first_commit_ts IS NOT NULL
AND
    t.first_view_ts IS NOT NULL
GROUP BY
    time_period
"""
control_results = pd.read_sql_query(query_control_results, conn)
control_results

,time_period,avg_diff
0,after,-112.710526
1,before,-99.464286


## 4. Close the connection.

In [18]:
conn.close()

## 5. Have the answer ready: "Did the hypothesis turn out to be true, and does the page affect students' behavior?"

Да, гипотеза подтвердилась. В тестовой группе разница между средним временем до дедлайна до первого просмотра страницы (-66.05 часов) и после (-99.52 часов) значительна. Это означает, что пользователи начали работать над лабораторными работами позже после того, как увидели новостную ленту.